In [1]:
import preprocess 
import pandas as pd

In [2]:
dfoff = pd.read_csv('../data/ccf_offline_stage1_train.csv')
dftest = pd.read_csv('../data/ccf_offline_stage1_test_revised.csv')
dfon = pd.read_csv('../data/ccf_online_stage1_train.csv')

In [3]:
# 预处理，部分特征处理
dfoff,weekdaycols = preprocess.preprocess(dfoff)
dftest,weekdaycols = preprocess.preprocess(dftest)

In [4]:
# 分割测试子集和训练子集
x_train,y_train,x_test,y_test = preprocess.split_t_v(dfoff,weekdaycols)